In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.tree import *
from sklearn.svm import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.model_selection import *
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.ensemble import *
import seaborn as sns

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/UNSW_NB15_training-set.csv',index_col='id')
train.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/UNSW_NB15_testing-set.csv',index_col='id')
test.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0


In [ ]:
data = pd.concat([train,test])

In [ ]:
data = data.replace({'-':np.nan})
data.loc[data.is_ftp_login>1,'is_ftp_login'] = 1
data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,NaN,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,NaN,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,NaN,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,NaN,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [ ]:
data['pkts'] = data['spkts'] - data['dpkts']
data['bytes'] = data['sbytes'] - data['dbytes']
data['loss'] = data['sloss'] + data['dloss']
data['inkpt'] = (data['sinpkt'] - data['dinpkt']) 
data['ttl'] = (data['sttl'] + data['dttl'])
data['load'] = (data['sload'] - data['dload'])
data['jit'] = (data['sjit'] + data['djit'])
data['tcpb'] = (data['stcpb'] - data['dtcpb'])
data['mean'] = (data['smean'] - data['dmean'])

In [ ]:
data.corrwith(data['label'])

dur                  0.029096
spkts               -0.043040
dpkts               -0.097394
sbytes               0.019376
dbytes              -0.060403
rate                 0.335883
sttl                 0.624082
dttl                 0.019369
sload                0.165249
dload               -0.352169
sloss                0.001828
dloss               -0.075961
sinpkt              -0.155454
dinpkt              -0.030136
sjit                -0.016436
djit                -0.048819
swin                -0.364877
stcpb               -0.266585
dtcpb               -0.263543
dwin                -0.339166
tcprtt              -0.024668
synack              -0.043250
ackdat               0.000817
smean               -0.028372
dmean               -0.295173
trans_depth          0.002246
response_body_len   -0.018930
ct_srv_src           0.246596
ct_state_ttl         0.476559
ct_dst_ltm           0.240776
ct_src_dport_ltm     0.318518
ct_dst_sport_ltm     0.371672
ct_dst_src_ltm       0.299609
is_ftp_log

In [ ]:
data.describe()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,pkts,bytes,loss,inkpt,ttl,load,jit,tcpb,mean
count,257673.000000,257673.000000,257673.000000,2.576730e+05,2.576730e+05,2.576730e+05,257673.000000,257673.000000,2.576730e+05,2.576730e+05,257673.000000,257673.000000,257673.000000,257673.000000,2.576730e+05,257673.000000,257673.000000,2.576730e+05,2.576730e+05,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,2.576730e+05,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,2.576730e+05,257673.000000,257673.000000,257673.000000,2.576730e+05,2.576730e+05,2.576730e+05,257673.000000
mean,1.246715,19.777144,18.514703,8.572952e+03,1.438729e+04,9.125391e+04,180.000931,84.754957,7.060869e+07,6.582143e+05,4.889317,6.743691,912.300834,98.915462,5.419373e+03,582.251456,121.753661,1.006120e+09,1.002295e+09,119.254629,0.046038,0.023652,0.022386,137.639027,121.649703,0.102242,1.968900e+03,9.383176,1.324978,6.050467,5.238271,4.032677,8.322964,0.012593,0.012850,0.132005,6.800045,9.121049,0.014274,0.639077,1.262441,-5.814336e+03,11.633008,813.385373,264.755888,6.995048e+07,6.001624e+03,3.825338e+06,15.989324
std,5.974305,135.947152,111.985965,1.737739e+05,1.461993e+05,1.603446e+05,102.488268,112.762131,1.857313e+08,2.412372e+06,65.574953,53.702222,6922.153239,1094.048691,4.903450e+04,3930.153369,127.367443,1.367795e+09,1.363877e+09,127.230477,0.092908,0.053856,0.045771,205.901118,254.041013,0.710593,4.962523e+04,10.829706,0.992300,8.173749,8.160822,5.831515,11.120754,0.111512,0.116421,0.681854,8.396266,10.874752,0.118618,0.480269,139.118847,2.259758e+05,85.457156,6893.202254,149.895360,1.859926e+08,5.112585e+04,1.155736e+09,340.812105
min,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,24.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,-9376.000000,-1.458683e+07,0.000000,-38443.245330,0.000000,-2.168261e+07,0.000000e+00,-4.271587e+09,-1447.000000
25%,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,3.078928e+01,62.000000,0.000000,1.231800e+04,0.000000e+00,0.000000,0.000000,0.008000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,57.000000,0.000000,0.000000,0.000000e+00,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,-3.200000e+01,0.000000,-0.174665,254.000000,-6.814781e+02,0.000000e+00,-3.240190e+05,-4.000000
50%,0.004285,4.000000,2.000000,5.280000e+02,1.780000e+02,2.955665e+03,254.000000,29.000000,7.439423e+05,1.747441e+03,0.000000,0.000000,0.381696,0.007000,6.736370e-01,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,73.000000,44.000000,0.000000,0.000000e+00,5.000000,1.000000,2.000000,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,4.000000,0.000000,1.000000,2.000000,1.140000e+02,0.000000,0.007000,254.000000,6.511949e+04,1.095445e+01,0.000000e+00,57.000000
75%,0.685777,12.000000,10.000000,1.362000e+03,1.064000e+03,1.250000e+05,254.000000,252.000000,8.000000e+07,2.210538e+04,3.000000,2.000000,58.094727,56.438859,2.787367e+03,119.712937,255.000000,2.007375e+09,1.992752e+09,255.000000,0.082082,0.036842,0.044665,100.000000,89.000000,0.000000,0.000000e+00,12.000000,2.000000,6.000000,4.000000,3.0000

In [ ]:
for i in ['pop3','ssh','dhcp','snmp','ssl','irc','radius']:
        data['service'][data.service == i ] = 'other_service'
for i in set(set(data.proto.unique()) - set(['tcp','udp','unas'])):
        data['proto'][data.proto == i ] = 'other_proto'
for i in ['ECO', 'REQ', 'RST', 'PAR', 'URN', 'no','ACC', 'CLO']:
        data['state'][data.state == i ] = 'other_state'

In [ ]:
categoricals = ['service','state','proto']
for i in categoricals:
    dummies = pd.get_dummies(data[i],drop_first= True,dummy_na=True,prefix=i)
    data    = pd.concat([data,dummies], axis=1)
    
data = data.drop([*categoricals,'attack_cat','dwin'],axis=1) # because mostly same as swin

In [ ]:
def FAR(y_true,y_pred):
    CM = confusion_matrix(y_true, y_pred)

    TN = CM[1][1]
    FN = CM[0][1]
    TP = CM[0][0]
    FP = CM[1][0]
    return FP/(FP+TN)

In [ ]:
def preprocess_final(X_train, X_test, y_train, y_test):


    scaler = QuantileTransformer()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def f_score(X_train, X_test, y_train, y_test):
    
    """
    XGB
    """
    model = XGBClassifier(silent=True,
                      booster = 'gbtree',
                     objective='binary:logistic',
                      n_estimators=5000, 
                        learning_rate = 0.05,
                      max_depth=7,
                        verbosity=0,
                     )
    eval_set = [(X_test, y_test)]
    eval_metric = ["logloss"]
    model.fit(X_train, y_train,early_stopping_rounds=100, eval_metric=eval_metric, eval_set=eval_set)

    """
    SVLinear tuning
    """
    # tuned_parameters = {
    #     'penalty':['l1','l2'],
    #     'C': [0.01,0.1,1, 10, 100],
    #     'loss': ['hinge','squared_hinge'],
    #     'multi_class':['ovr', 'crammer_singer']
    # }
    # model = GridSearchCV(LinearSVC(max_iter = 10), tuned_parameters,n_jobs=-1,verbose=3,cv=3)
    # model.fit(X_train,y_train)
    # print(model.best_params_)

    
    """
    SV Linear
    """
    # model = LinearSVC(max_iter = 2000,C= 0.1, loss= 'squared_hinge', multi_class='crammer_singer')
    # model = LinearSVC(max_iter = 2000,)
    # model.fit(X_train,y_train)
    """
    LGBM
    """

    #model = lgb.LGBMClassifier()
    #model.fit(X_train,y_train)
    
    
    yxgb=model.predict(X_test)
    far = FAR(y_true=y_test,y_pred=yxgb)
    acc = roc_auc_score(y_true=y_test,y_score=yxgb,)
    print(classification_report(y_true=y_test,y_pred=yxgb))
    print(confusion_matrix(y_true=y_test,y_pred=yxgb))
    return (far,acc)

In [ ]:
def category(X_train, X_test, y_train, y_test):
        
    X_train, X_test, y_train, y_test = preprocess_final(X_train, X_test, y_train, y_test)
    score_f,score_a  = f_score(X_train, X_test, y_train, y_test)
    return score_f,score_a

In [ ]:
y  = data['label']
X  = data.drop(['label'],axis=1)

train_size = len(train)

X_train, X_test = X[:][:train_size],X[:][train_size:]
y_train, y_test = y[:][:train_size],y[:][train_size:]

s = X_train.skew().reset_index().rename(columns = {0:'skew'})

pos = list(s[s['skew']>=1]['index'].values)
neg = list(s[s['skew']<=-1]['index'].values)

X_train[pos] = (abs(X_train[pos])+1).apply(np.log2)
X_train[neg] = (X_train[neg])**3

X_test[pos] = (abs(X_test[pos])+1).apply(np.log2)
X_test[neg] = (X_test[neg])**3

result1, result3 = category(X_train, X_test, y_train, y_test)
print(f"FAR : {result1}, Auc: {result3}")

[0]	validation_0-logloss:0.65820
[1]	validation_0-logloss:0.62655
[2]	validation_0-logloss:0.59818
[3]	validation_0-logloss:0.57227
[4]	validation_0-logloss:0.54869
[5]	validation_0-logloss:0.52694
[6]	validation_0-logloss:0.50718
[7]	validation_0-logloss:0.48908
[8]	validation_0-logloss:0.47226
[9]	validation_0-logloss:0.45746
[10]	validation_0-logloss:0.44321
[11]	validation_0-logloss:0.43054
[12]	validation_0-logloss:0.41865
[13]	validation_0-logloss:0.40730
[14]	validation_0-logloss:0.39674
[15]	validation_0-logloss:0.38682
[16]	validation_0-logloss:0.37774
[17]	validation_0-logloss:0.36913
[18]	validation_0-logloss:0.36101
[19]	validation_0-logloss:0.35370
[20]	validation_0-logloss:0.34670
[21]	validation_0-logloss:0.34008
[22]	validation_0-logloss:0.33416
[23]	validation_0-logloss:0.32820
[24]	validation_0-logloss:0.32269
[25]	validation_0-logloss:0.31798
[26]	validation_0-logloss:0.31309
[27]	validation_0-logloss:0.30901
[28]	validation_0-logloss:0.30473
[29]	validation_0-loglos

# Reduced Features

In [ ]:
# y  = data['label']
# X  = data.drop(['label'],axis=1)

# train_size = len(train)

# X_train, X_test = X[:][:train_size],X[:][train_size:]
# y_train, y_test = y[:][:train_size],y[:][train_size:]

# s = X_train.skew().reset_index().rename(columns = {0:'skew'})
# pos = list(s[s['skew']>=1]['index'].values)
# neg = list(s[s['skew']<=-1]['index'].values)

# X_train[pos] = (abs(X_train[pos])+1).apply(np.log2)
# X_train[neg] = (X_train[neg])**3

# X_test[pos] = (abs(X_test[pos])+1).apply(np.log2)
# X_test[neg] = (X_test[neg])**3


# from sklearn.ensemble import RandomForestClassifier

# sel = SelectFromModel(RandomForestClassifier())
# sel.fit(X_train,y_train)
# features = X_train.columns[(sel.get_support())]
# print(len(features))

# X_train = X_train.filter(items=features)
# X_test = X_test.filter(items=features)

# result1, result3 = category(X_train, X_test, y_train, y_test)
# print(f"FAR : {result1}, Auc: {result3}")

24
              precision    recall  f1-score   support

           0       0.97      0.56      0.71     37000
           1       0.73      0.98      0.84     45332

    accuracy                           0.79     82332
   macro avg       0.85      0.77      0.77     82332
weighted avg       0.84      0.79      0.78     82332

[[20600 16400]
 [  733 44599]]
FAR : 0.016169593223330097, Auc: 0.7702935817667134
